In [3]:
import numpy as np
import pandas as pd
import glob
from tqdm.notebook import tqdm, trange
from scipy import signal, stats
from scipy.fft import fft
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score

# Internal functions

In [6]:
def segmentation(df, overlap_rate, time_window):
    
    # make a list for segment window and its label
    seg_data = []

    #convert overlap rate to step for sliding window
    overlap = int((1 - overlap_rate)*time_window)
    
    # interpolate
    df = df.interpolate().ffill().fillna(0)
    #segment
    for i in range(0, len(df)-time_window+1, overlap):
        seg_data.append(df.loc[i:i+time_window-1, :].copy().reset_index(drop=True))
        
    return seg_data

In [7]:
def rename_columns(df):
    df.columns = [
        "FH_X", "FH_Y", "FH_Z",     #1
        "TH_X", "TH_Y", "TH_Z",     #2
        "RH_X", "RH_Y", "RH_Z",     #3
        "RS_X", "RS_Y", "RS_Z",     #4
        "RO_X", "RO_Y", "RO_Z",     #5
        "RE_X", "RE_Y", "RE_Z",     #6
        "RW_X", "RW_Y", "RW_Z",     #7
        "LS_X", "LS_Y", "LS_Z",     #8
        "LE_X", "LE_Y", "LE_Z",     #9
        "LW_X", "LW_Y", "LW_Z",     #10
        "RA_X", "RA_Y", "RA_Z",     #11
        "LA_X", "LA_Y", "LA_Z",     #12
        "VS_X", "VS_Y", "VS_Z",     #13
        "subject_id", "activity",   # Other columns
    ]
    return df

# Feature related functions

In [9]:
def get_speed_acc(x_data):
    x_data = x_data.drop(columns=["activity", "subject_id"])
    speed = x_data.diff().fillna(0)
    acc = speed.diff().fillna(0)
    speed.columns = [f"{col}_speed" for col in speed.columns]
    acc.columns = [f"{col}_acc" for col in acc.columns]
    return speed, acc


def get_speed_acc_jerk(x_data):
    x_data = x_data.drop(columns=["activity", "subject_id"])
    speed = x_data.diff().fillna(0)
    acc = speed.diff().fillna(0)
    jerk = acc.diff().fillna(0)
    speed.columns = [f"{col}_speed" for col in speed.columns]
    acc.columns = [f"{col}_acc" for col in acc.columns]
    jerk.columns = [f"{col}_jerk" for col in acc.columns]
    return speed, acc, jerk

In [10]:
def joint_distance(x_data, joint1, joint2):
    """
    returns the distance between two joints. 
    """
    x1, y1, z1 = x_data[f"{joint1}_X"], x_data[f"{joint1}_Y"], x_data[f"{joint1}_Z"]
    x2, y2, z2 = x_data[f"{joint2}_X"], x_data[f"{joint2}_Y"], x_data[f"{joint2}_Z"]
    distance = np.sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)
    return distance

def get_all_joint_distances(x_data):
    """
    calculates all the necessary joint distances from the `x_data`, 
    adds columns to it and returns the modified `x_data`.
    the two joints should not be essentially consecutive, 
    because the distance between two consecutive joints is always constant.
    For example, distance between left_wrist and left_elbow is always constant.
    """
    # joints
    # Front head        ->  left shoulder       (1->8)
    x_data["dist_FH_LS"] = joint_distance(x_data, "FH", "LS")
    # Front head        ->  right shoulder      (1->4)
    x_data["dist_FH_RS"] = joint_distance(x_data, "FH", "RS")
    # left shoulder     ->  left wrist          (8->10)
    x_data["dist_LS_LW"] = joint_distance(x_data, "LS", "LW")
    # right shoulder    ->  right wrist         (4->7)
    x_data["dist_RS_RW"] = joint_distance(x_data, "RS", "RW")
    # v sacral          ->  left elbow          (13->9)
    x_data["dist_VS_LE"] = joint_distance(x_data, "VS", "LE")
    # v sacral          ->  right elbow         (13->6)
    x_data["dist_VS_RE"] = joint_distance(x_data, "VS", "RE")
    # v sacral          ->  left wrist          (13->10)
    x_data["dist_VS_LW"] = joint_distance(x_data, "VS", "LW")
    # v sacral          ->  right wrist         (13->7)
    x_data["dist_VS_RW"] = joint_distance(x_data, "VS", "RW")
    # v sacral          ->  rear head           (13->3)
    x_data["dist_VS_RH"] = joint_distance(x_data, "VS", "RH")
    # v sacral          ->  top head            (13->2)
    x_data["dist_VS_TH"] = joint_distance(x_data, "VS", "TH")
    # left wrist        ->  right wrist         (10->7)
    x_data["dist_LW_RW"] = joint_distance(x_data, "LW", "RW")
    # left asis         ->  left wrist          (12->10)
    x_data["dist_LA_LW"] = joint_distance(x_data, "LA", "LW")
    # right asis        ->  right wrist         (11->7)
    x_data["dist_RA_RW"] = joint_distance(x_data, "RA", "RW")
    # left wrist        ->  top head            (10->2)
    x_data["dist_LW_TH"] = joint_distance(x_data, "LW", "TH")
    # right wrist       ->  top head            (7->2)
    x_data["dist_RW_TH"] = joint_distance(x_data, "RW", "TH")
    # top head          ->  left asis           (2->12)
    x_data["dist_TH_LA"] = joint_distance(x_data, "TH", "LA")
    return x_data

In [11]:
def joint_angle(x_data, joint1, joint2, joint3):
    x1, y1, z1 = x_data[f"{joint1}_X"], x_data[f"{joint1}_Y"], x_data[f"{joint1}_Z"]
    x2, y2, z2 = x_data[f"{joint2}_X"], x_data[f"{joint2}_Y"], x_data[f"{joint2}_Z"]
    x3, y3, z3 = x_data[f"{joint3}_X"], x_data[f"{joint3}_Y"], x_data[f"{joint3}_Z"]
    v1 = np.array([x2-x1, y2-y1, z2-z1]).T
    v2 = np.array([x3-x2, y3-y2, z3-z2]).T
    v1_unit = v1/np.expand_dims(np.linalg.norm(v1, axis=1), axis=1)
    v2_unit = v2/np.expand_dims(np.linalg.norm(v2, axis=1), axis=1)
    angle = np.arccos(np.sum(v1_unit*v2_unit, axis=1)) # dot multiplication
    return angle

def get_all_joint_angles(x_data):
    # joints
    # left shoulder     ->  left elbow      ->  left wrist      (8->9->10)
    x_data["angle_LS_LE_LW"] = joint_angle(x_data, "LS", "LE", "LW")
    # right shoulder    ->  right elbow     ->  right wrist     (4->6->7)
    x_data["angle_RS_RE_RW"] = joint_angle(x_data, "RS", "RE", "RW")
    # right shoulder    ->  left shoulder   ->  front head      (4->8->1)
    x_data["angle_RS_LS_FH"] = joint_angle(x_data, "RS", "LS", "FH")
    # right shoulder    ->  left shoulder   ->  left elbow      (4->8->9)
    x_data["angle_RS_LS_LE"] = joint_angle(x_data, "RS", "LS", "LE")
    # left shoulder     ->  right shoulder  ->  right elbow     (8->4->6)
    x_data["angle_LS_RS_RE"] = joint_angle(x_data, "LS", "RS", "RE")
    # v sacral          ->  right offset    ->  rear head       (13->5->3)
    x_data["angle_VS_RO_RH"] = joint_angle(x_data, "VS", "RO", "RH")
    # vsacral           ->  top head        ->  front head      (13->2->1)
    x_data["angle_VS_TH_FH"] = joint_angle(x_data, "VS", "TH", "FH")
    # v sacral          ->  left shoulder   ->  left elbow      (13->8->9)
    x_data["angle_VS_LS_LE"] = joint_angle(x_data, "VS", "LS", "LE")
    # v sacral          ->  right shoulder  ->  right elbow     (13->4->6)
    x_data["angle_VS_RS_RE"] = joint_angle(x_data, "VS", "RS", "RE")
    # left asis         ->  left shoulder   ->  left elbow      (12->8->9)
    x_data["angle_LA_LS_LE"] = joint_angle(x_data, "LA", "LS", "LE")
    # right asis        -> right shoulder   ->  right elbow     (11->4->6)
    x_data["angle_RA_RS_RE"] = joint_angle(x_data, "RA", "RS", "RE")
    return x_data

In [13]:
def plane_angles(x_data, joint1, joint2):
    x1, y1, z1 = x_data[f"{joint1}_X"], x_data[f"{joint1}_Y"], x_data[f"{joint1}_Z"]
    x2, y2, z2 = x_data[f"{joint2}_X"], x_data[f"{joint2}_Y"], x_data[f"{joint2}_Z"]
    v = np.array([x2-x1, y2-y1, z2-z1]).T
    vx = np.array([1, 0, 0])
    vy = np.array([0, 1, 0])
    vz = np.array([0, 0, 1])
    v_unit = v/np.expand_dims(np.linalg.norm(v, axis=1), axis=1)
    angle_x = np.arccos(np.sum(v_unit*vx, axis=1)) 
    angle_y = np.arccos(np.sum(v_unit*vy, axis=1)) 
    angle_z = np.arccos(np.sum(v_unit*vz, axis=1)) 
    return angle_x, angle_y, angle_z

def get_all_angles_with_plane(x_data):
    # joints
    # arm angles: left shoulder     ->  left elbow 
    x_data["angle_x_LS_LE"], x_data["angle_y_LS_LE"], x_data["angle_z_LS_LE"] = plane_angles(x_data, "LS", "LE")
    # forearm angles: left elbow    ->  left  wrist
    x_data["angle_x_LE_LW"], x_data["angle_y_LE_LW"], x_data["angle_z_LE_LW"] = plane_angles(x_data, "LE", "LW")
    # arm angles: right shoulder    ->  right elbow 
    x_data["angle_x_RS_RE"], x_data["angle_y_RS_RE"], x_data["angle_z_RS_RE"] = plane_angles(x_data, "RS", "RE")
    # forearm angles: right elbow   ->  right  wrist
    x_data["angle_x_RE_RW"], x_data["angle_y_RE_RW"], x_data["angle_z_RE_RW"] = plane_angles(x_data, "RE", "RW")
    # backbone angles: v sacral     ->  rear head
    x_data["angle_x_VS_RH"], x_data["angle_y_VS_RH"], x_data["angle_z_VS_RH"] = plane_angles(x_data, "VS", "RH")
    return x_data

In [16]:
def num_zero_crossings(series):
    arr = series.to_numpy()
    return ((arr[:-1] * arr[1:]) < 0).sum()

def get_peaks(series):
    peaks, _ = signal.find_peaks(series)
    num_peaks = len(peaks)
    if not num_peaks:
        return 0, 0, 0, 0
    mean_diff = np.diff(peaks).mean() if num_peaks>1 else 0
    mean_height = series[peaks].mean()
    std_height = series[peaks].std()
    return num_peaks, mean_diff, mean_height, std_height


# Data Operation functions

In [ ]:
def dataloader(overlap, window_size, verbose=True):
    if verbose:
        print("loading the data...", end="\t")
    data_list = []
    file_lengths = {1: [], 2: [], 3: []}
    files = tqdm(glob.glob("../TrainData/*/*/*.csv")) if verbose else glob.glob("../TrainData/*/*/*.csv")
    for file in files:
        tempdf = pd.read_csv(file)
        tempdf = rename_columns(tempdf)
        segmented_data = segmentation(tempdf, overlap, window_size)
        if len(segmented_data)>0:
            person = segmented_data[0].reset_index(drop=True).loc[0, "subject_id"]
            file_lengths[person].append(len(segmented_data))   
        data_list.extend(segmented_data)
    return data_list, file_lengths


def feature_extractor(data_list, verbose=True):
    if verbose:
        print(f"extracting the features...", end="  ")
    X, y = {1:[], 2:[], 3:[]}, {1:[], 2:[], 3:[]}
    num_range = trange(0,len(data_list)) if verbose else range(0,len(data_list))
    for j in num_range:
        #extract only xyz columns
        person = data_list[j].loc[0, "subject_id"]
        x_data = data_list[j].drop(columns=["subject_id","activity"])
        X[person].append(get_features(x_data))
        y[person].append(data_list[j].reset_index(drop=True).loc[0, "activity"])
    return X, y

def get_processed_dataset(overlap_rate, window_size, verbose=True):
    data_list, file_lengths = dataloader(overlap_rate, window_size, verbose=verbose)
    stream_list = []
    for df in data_list:
        stream_list.append(get_streams(df))
    X, y = feature_extractor(stream_list, verbose=verbose)
    return X, y, file_lengths


def model_evaluator(model, X, y, file_lengths, n_repeats=7, voting=True, verbose=True):
    scores = []
    num_range = trange(n_repeats) if verbose else range(n_repeats)
    for _ in num_range:
        for p1, p2, p3 in [(1,2,3), (2,3,1), (3,1,2)]:
            X_test, y_test = X[p1], y[p1]
            X_train = X[p2] + X[p3]
            y_train = y[p2] + y[p3]
            # print(f"training model for person {p1}/3...", end="\t")
            model.fit(X_train, y_train)
            pred = model.predict(X_test)
            if voting:
                filtered_pred = majority_voting(pred, file_lengths[p1])
                scores.append(accuracy_score(y_test, filtered_pred))
            else:
                scores.append(accuracy_score(y_test, pred))
    if verbose:
        print(f"Mean Score: {np.mean(scores)}")
        print(f"Std Score: {np.std(scores)}")
        print(f"Min Score: {np.min(scores)}")
        print(f"Max Score: {np.max(scores)}")
    return scores

# Functions to be tuned

In [ ]:
def get_features(x_data):
    features = []
    cols = x_data.columns.tolist()
    #Calculate features (STD, Average, Max, Min, Median, Variance) for each data columns X Y Z 
    for k in cols:
        # time domain features
        features.append(x_data[k].std(ddof=0))          # std
        features.append(np.average(x_data[k]))          # average
        features.append(np.max(x_data[k]))              # max
        features.append(np.min(x_data[k]))              # min
        features.append(np.median(x_data[k]))           # median
        features.append(np.var(x_data[k]))              # variance
        # extra time features
        features.append(num_zero_crossings(x_data[k]))  # num of zero crossing
        features.extend(get_peaks(x_data[k]))           # peak related features
        features.append((x_data[k]**2).sum())           # energy
        features.append(stats.iqr(x_data[k]))           # inter quartile range

        # freq domain features
        fd = np.abs(fft(np.array(x_data[k])))**2        # freq domain signal
        features.append(stats.skew(fd))                 # skew
        features.append(stats.kurtosis(fd))             # kurtosis
        # extra freq domain features
        features.append(fd.std(ddof=0))                 # std
        features.append(np.average(fd))                 # average
        features.append(np.max(fd))                     # max
        features.append(np.min(fd))                     # min
        features.append(np.median(fd))                  # median
        features.append(stats.iqr(fd))           # inter quartile range
    return features


def get_streams(x_data):
    speed, acc = get_speed_acc(x_data)
    x_data = pd.concat([x_data, speed, acc], axis=1)
    x_data = get_all_joint_angles(x_data)
    x_data = get_all_joint_distances(x_data)
    x_data = get_all_angles_with_plane(x_data)
    return x_data

# driver code

In [ ]:
# Need to change get_streams, get_features

# load the data into features
overlap_rate, window_size = 0.75, 3000
X, y, file_lengths = get_processed_dataset(overlap_rate, window_size)

In [ ]:
model = ETC(1800, criterion="gini", min_samples_split=4, max_depth=12, n_jobs=-1)
scores = model_evaluator(model, X, y, file_lengths, n_repeats=10, voting=True, verbose=True)

In [17]:
# data_list = []
# for file in tqdm(glob.glob("../TrainData/*/*/*.csv")):
#     tempdf = pd.read_csv(file)
#     tempdf = rename_columns(tempdf)
#     data_list.extend(segmentation(tempdf, 0.5, 500))

# stream_list = []
# for df in tqdm(data_list):
#     stream_list.append(get_streams(df))


# features_list = []
# label_list = []
# for j in trange(0,len(stream_list)):
#     #extract only xyz columns
#     x_data = stream_list[j].drop(columns=["subject_id","activity"])

#     #Get features and label for each elements
#     features_list.append(get_features(x_data))
#     label_list.append(stream_list[j].iloc[0, stream_list[j].columns.tolist().index("activity")])

  0%|          | 0/3573 [00:00<?, ?it/s]

In [18]:
rf = RFC(300, n_jobs=-1)

In [19]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
n_scores = cross_val_score(rf, features_list, label_list, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise', verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.1min finished


In [20]:
n_scores.mean(), n_scores

(0.8430089354177424,
 array([0.79050279, 0.84078212, 0.82681564, 0.85994398, 0.86554622,
        0.86834734, 0.82352941, 0.85434174, 0.85434174, 0.84593838]))